
# Fine-Tuning Lab

Welcome to the fine-tuning lab! By the end of this lab, you will have

1. Loaded a pretrained network and ask it for predictions
2. Trained a logistic model on top of a frozen pretrained network to predict cats from dogs
3. Fine-tuned a pretrained network to predict cats from dogs
4. Performed error analysis to get insight into next steps to take

Let's get started!

---

# Pretrained Networks

In this section, we will practice using a pretrained model, in particular `VGGNet`. `VGGNet` won the 2013 [*ImageNet Large Scale Visual Recognition Challenge (ILSVRC)*](http://image-net.org/challenges/LSVRC/2013/). After loading `VGGNet`, we will have it to make predictions on our images. We will then modify it so that it is tailored to the cats vs. dogs dataset.

# Loading a Pre-Trained `VGGNet` in `keras`

![VGGNet](http://machinethink.net/images/vggnet-convolutional-neural-network-iphone/VGGNet@2x.png)

Keras provides a simple and easy way of loading in a pretrained `VGGNet` for us to play around with.

## Tasks

- Instantiate a `keras.applications.VGG16` model with `ImageNet` pretrained weights
- Use `keras.models.Model.summary()` to visualize the architecture

## Hints

- [Documentation](https://keras.io/applications/#vgg16) for `keras.applications.VGG16`
- Check out the example **Extract features with VGG16** on the `keras` [applications](https://keras.io/applications/) page for how to load a `VGG16` model with pretrained `ImageNet` weights

## Notes

- The first time you instantiate a `keras.applications.vgg16.VGG16` with ImageNet weights keras will download the weights which might take some time so be patient!

## Task

- Download the file `CatsVDogs.p` and put it in an accessible directory and load the data into memory

In [1]:
import pickle

[imgs, y] = pickle.load(open('data/CatsVDogs.p', 'rb'))

# Preprocessing Images

Even though the cats vs. dogs dataset has different labels than those found in `ImageNet`, we can still ask `VGGNet` to make predictions and output the most likely `ImageNet` classes. But before doing this, we need to perform the necessary preprocessing the VGGNet expects.

### Tasks

- Convert `imgs` to a `4D` `numpy` array and save it to a variable named `X`
- Perform the preprocessing on `X` that `VGGNet16` expects
- Display five prepocessed images

### Hints

- Use `keras.applications.vgg16.preprocess_input()` to zero-center a 4D tensor of images with respect to the mean RGB value in `ImageNet`

## Question

- What does `keras.applications.vgg16.preprocess_input()` do?

## Hint

- Look at how and where [`keras.applications.vgg16.preprocess_input()`](https://github.com/fchollet/keras/blob/master/keras/applications/vgg16.py#L27) is defined

## Answer

- The color channels are shifted and then the mean pixel intensity is subtracted channel-wise.

# `VGGNet` Predictions

Let's make some predictions with our VGGNet.

## Tasks

- Use `keras.applications.vgg16.predict()` to make predictions on the first four images of `X`
- Use `keras.applications.vgg16.decode_predictions()` to recover the object class names from the `VGGNet` predictions
- Display the first four images along with `VGGNet`'s predictions

## Hints

- Look at the `keras` [applications](https://keras.io/applications/) page for an example of making predictions with `VGGNet`

We see that `VGGNet` does a fairly good job at predicting sensibly classes. However, a lot of modeling effort is wasted by predicting probabilities for 1,000 classes when all we care about are *two* classes (i.e. cat and dog). In other words, `VGGNet` is not tailored to our specific dataset.

# Training a Classifier with `VGGNet` Features

Let's tailor `VGGNet` to the cats vs. dogs dataset by treating `VGGNet` as a black-box feature extractor train a logistic classifier on top.

### Tasks

- Replace the 1000-unit Dense+Softmax layer on `keras.applications.vgg16` with a `1`-unit Dense+Sigmoid layer
- Use `keras.models.Model.summary()` to ensure that you've successfully replaced the `1,000`-way softmax layer with a dense layer followed by a sigmoid activation

### Hints

- Look at the **Fine-tune InceptionV3 on a new set of classes** example on the `keras` [applications](https://keras.io/applications/) page for an example of how to modify the final layers of a `keras.models.Model`

# Freeze Base Model

We want to treat `VGGNet` as a fixed feature extractor and optimize only the weights in the final Dense layer.

## Tasks

- Set the `trainable` attribute on every `keras.layers.Layer` to `False` for every layer except the last dense layer in the `keras.models.Model`
- Print out the name of each `keras.layers.Layer` with the value of its `trainable` attribute to ensure you've been successful

## Hints

- The `keras.models.Model.layers` attribute provides access the layers in a `keras` model
- The `keras.layers.Layer.name` attribute contains the name of a layer
- `keras.layers.Layer.trainable = False` freezes that layer during training

## Reminder

- Make sure to compile your model after freezing weights for it to really take effect!

# Train Dense Layer on Cats vs. Dogs Dataset

It's time we learn the parameters of our newly defined dense layer on the cats vs. dogs dataset!

## Tasks

- Compile the `keras.models.Model`
- Call `keras.models.Model.fit()` with the images and labels for 1 epoch and with a `validation_split=0.1`

## Suggestions

- Kill the training after it finishes the first batch (otherwise it will take forever and our concern here is not overall accuracy but learning)

# Make Predictions

Given a new image, the `keras.models.Model` should now output a single number $p(y = \text{dog}\ |\ x)$.

## Tasks

- Use `keras.models.Model.predict()` on the first four images of `X_tf`
- Display the images along with their predictions

Using `VGGNet` as a black-box feature extractor gets us respectible performance. However, we can do better by leveraging supervised *fine-tuning*, a type of *transfer learning*.

# Fine-Tuning `VGGNet` on Cats vs. Dogs Dataset

In addition to learning the final Dense+Sigmoid layer, we can also update the last few layers of `VGGNet` during training through a process called *fine-tuning*. This process is illustrated in the following figure.

![Fine-Tuning](https://blog.keras.io/img/imgclf/vgg16_modified.png)

# Set Up `VGGNet` for Fine-Tuning

## Tasks

- Unfreeze the penultimate dense layer along with the final conv block
- Print out each layer name with the value of its `trainable` attribute to ensure you've successfully unfrozen the last conv block plus the final two dense layers

## Hints

- Use `keras.models.Model.summary()` to determine the names for the last two `Dense` layers
- Use `keras.models.Model.get_layer()` to get references to these layers
- `keras.layers.Layer.trainable = True` unfreezes a `keras.layers.Layer`

# Fine-Tune `VGGNet`

Now that we have unfrozen the last two `Dense` layers and the final conv block we can optimize these weights for even better performance.

## Tasks

- `keras.models.Model.compile()` the model with a stochastic gradient descent optimizer with a small learning rate (e.g. $10^{-4}$)
- `keras.models.Model.fit()` the `keras.models.Model` to the image data with a `validation_split=1` for one epoch

## Suggestions

- Kill the training after a minibatch update since it takes forever on a CPU

### Make Predictions with Fine-Tuned `VGGNet`

### Tasks

- `keras.applications.vgg16.predict()` the labels of the first four images in the cats vs. dogs dataset
- `IPython.display.display()` each image with its prediction

# Bonus Activities

---

# Error Analysis

Now that we have a reasonably performing image classifier, what's next? A good idea is to perform *error analysis* to systemtically discover which types of errors your model is making the most.

## Task

- Make predictions on the first 128 images

## Hint

- Recall that cats have labels of `0`, while dogs have labels of `1`

## Admission

- In a real-world setting you would make predictions on a held-out test set. However, we have not gone through the trouble of creating one and hence using the training set will do this one time.

## Task

- Display five images which your model made the correct prediction

## Task

- Display five images which your model made an incorrect prediction

## Task

- Display the top five cat images that your model is very sure contains a cat along with its confidence for each image

## Task

- Display the top five cat images that your model is very sure contains a dog along with its confidence for each image

## Task

- Display the top five images which confuse your model the most along with its confidence for each image